In [2]:
library(rgdal)
library(spdep)
library(plm)
library(lmtest)
library(tidyr)

Warning message:
: package ‘rgdal’ was built under R version 3.2.5Loading required package: sp
Warning message:
: package ‘sp’ was built under R version 3.2.5rgdal: version: 1.1-10, (SVN revision 622)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 1.11.4, released 2016/01/25
 Path to GDAL shared files: /Library/Frameworks/R.framework/Versions/3.2/Resources/library/rgdal/gdal
 Loaded PROJ.4 runtime: Rel. 4.9.1, 04 March 2015, [PJ_VERSION: 491]
 Path to PROJ.4 shared files: /Library/Frameworks/R.framework/Versions/3.2/Resources/library/rgdal/proj
 Linking to sp version: 1.2-3 
Warning message:
: package ‘spdep’ was built under R version 3.2.5Loading required package: Matrix
Loading required package: Formula
Loading required package: zoo
Warning message:
: package ‘zoo’ was built under R version 3.2.5
Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Warning message:
: package

In [16]:
library(splm)

Warning message:
: package ‘splm’ was built under R version 3.2.5

In [3]:
setwd("~/Desktop")
chiFR<-readOGR(".","FA-Finalff")
head(chiFR@data)

OGR data source with driver: ESRI Shapefile 
Source: ".", layer: "FA-Finalff"
with 790 features
It has 62 fields


,AREAKEY,geoid10,STATE,PLACE,CBSA,ZCTA5,commarea,TRCTPOP0,TRCTPOP1,LINC14,⋯,LostP,BkRisk,NBkRisk,WhtRisk,FAChng,HispRisk,NetFAChng,BWht,BBlk,CATEGORIES
0,17031010100,17031010100,17,14000,16980,60626,1,5294,4854,4.65153,⋯,1,9,9,9,0,9,0,9,9,1
1,17031010201,17031010201,17,14000,16980,60645,1,6308,6450,4.6366985,⋯,1,9,9,9,0,9,0,9,1,1
2,17031010202,17031010202,17,14000,16980,60626,1,4398,2818,4.6694378,⋯,1,9,0,9,0,9,0,9,9,1
3,17031010300,17031010300,17,14000,16980,60626,1,6649,6236,4.6844055,⋯,1,9,0,0,0,9,0,9,9,2
4,17031010400,17031010400,17,14000,16980,60626,1,5325,5042,4.86623,⋯,1,9,0,0,0,9,0,9,9,2
5,17031010501,17031010501,17,14000,16980,60626,1,3835,4091,4.8302485,⋯,1,9,0,9,0,9,0,1,9,2


In [4]:
#Covert GeoID from factor to number that's digestable
geoid10<-as.numeric(as.character(chiFR@data$geoid10))

### Convert from wide to long format. There should two rows for each census tract for each time period

In [5]:
time1<- cbind(as.numeric(as.character(chiFR@data$geoid10)), chiFR@data$F_1,
              chiFR@data$LINC00, chiFR@data$RISK0, chiFR@data$Blk00D,
              chiFR@data$Wht00D,chiFR@data$Hisp00D,chiFR@data$FW1,2010)
colnames(time1)<- c("geoid10","FA","LINC","FRisk","BlkD","WhtD","HispD","FA_W","Year")
time1

geoid10,FA,LINC,FRisk,BlkD,WhtD,HispD,FA_W,Year
17031010100,-4.0866947,4.4766143,0,1,0,0,-4.1709978,2010
17031010201,-4.2259762,4.5345972,0,0,0,0,-3.9943778,2010
17031010202,-4.2311308,4.5345972,0,0,0,0,-4.1039396,2010
17031010300,-4.1108648,4.6227837,0,0,1,0,-3.9937516,2010
17031010400,-3.8139381,4.5746562,0,0,1,0,-3.919983,2010
17031010501,-3.6753876,4.4895788,0,0,0,0,-3.9560764,2010
17031010502,-3.7427297,4.4895788,0,0,1,0,-3.912858,2010
17031010503,-4.0053332,4.4895788,0,0,1,0,-3.9398047,2010
17031010600,-4.156773,4.5928538,0,0,0,0,-4.0185235,2010
17031010701,-4.0447435,4.5750608,0,0,0,0,-3.9833628,2010


In [6]:
time2<- cbind(as.numeric(as.character(chiFR@data$geoid10)), chiFR@data$F_2,
              chiFR@data$LINC14, chiFR@data$Risk1, chiFR@data$Blk14D,
              chiFR@data$Wht14D,chiFR@data$Hisp14D,chiFR@data$FW2,2014)
colnames(time2)<- c("geoid10","FA","LINC","FRisk","BlkD","WhtD","HispD","FA_W","Year")
time2

geoid10,FA,LINC,FRisk,BlkD,WhtD,HispD,FA_W,Year
17031010100,-4.0490105,4.65153,0,0,0,0,-4.0604156,2014
17031010201,-4.2356443,4.6366985,0,1,0,0,-3.9305237,2014
17031010202,-4.0378166,4.6694378,0,0,0,0,-4.0661337,2014
17031010300,-4.0830146,4.6844055,0,0,1,0,-3.9484348,2014
17031010400,-3.7902213,4.86623,0,0,1,0,-3.8771362,2014
17031010501,-3.7034517,4.8302485,0,0,1,0,-3.9147683,2014
17031010502,-3.6999515,4.6750539,0,0,1,0,-3.8445135,2014
17031010503,-3.7984955,4.3924859,0,0,1,0,-3.9107924,2014
17031010600,-4.0858857,4.7844889,0,0,1,0,-3.9500298,2014
17031010701,-3.9190241,4.732434,0,0,1,0,-4.0354698,2014


In [7]:
test<- rbind(data.frame(time1),data.frame(time2))
head(test)


,geoid10,FA,LINC,FRisk,BlkD,WhtD,HispD,FA_W,Year
1,1.703101e+10,-4.086695e+00,4.476614e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-4.170998e+00,2.010000e+03
2,1.703101e+10,-4.225976e+00,4.534597e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.994378e+00,2.010000e+03
3,1.703101e+10,-4.231131e+00,4.534597e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-4.103940e+00,2.010000e+03
4,1.703101e+10,-4.110865e+00,4.622784e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-3.993752e+00,2.010000e+03
5,1.703101e+10,-3.813938e+00,4.574656e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-3.919983e+00,2.010000e+03
6,1.703101e+10,-3.675388e+00,4.489579e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.956076e+00,2.010000e+03


In [8]:
test$TractYear<- (test$geoid10 * test$Year) 
head(test)

,geoid10,FA,LINC,FRisk,BlkD,WhtD,HispD,FA_W,Year,TractYear
1,1.703101e+10,-4.086695e+00,4.476614e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-4.170998e+00,2.010000e+03,3.423233e+13
2,1.703101e+10,-4.225976e+00,4.534597e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.994378e+00,2.010000e+03,3.423233e+13
3,1.703101e+10,-4.231131e+00,4.534597e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-4.103940e+00,2.010000e+03,3.423233e+13
4,1.703101e+10,-4.110865e+00,4.622784e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-3.993752e+00,2.010000e+03,3.423233e+13
5,1.703101e+10,-3.813938e+00,4.574656e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-3.919983e+00,2.010000e+03,3.423233e+13
6,1.703101e+10,-3.675388e+00,4.489579e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.956076e+00,2.010000e+03,3.423233e+13


In [9]:
test$YearD<- ifelse(test$Year == "2010", 0, 1)
test$FD<- test$YearD * test$FRisk

In [10]:
head(test)

,geoid10,FA,LINC,FRisk,BlkD,WhtD,HispD,FA_W,Year,TractYear,YearD,FD
1,1.703101e+10,-4.086695e+00,4.476614e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-4.170998e+00,2.010000e+03,3.423233e+13,0.000000e+00,0.000000e+00
2,1.703101e+10,-4.225976e+00,4.534597e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.994378e+00,2.010000e+03,3.423233e+13,0.000000e+00,0.000000e+00
3,1.703101e+10,-4.231131e+00,4.534597e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-4.103940e+00,2.010000e+03,3.423233e+13,0.000000e+00,0.000000e+00
4,1.703101e+10,-4.110865e+00,4.622784e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-3.993752e+00,2.010000e+03,3.423233e+13,0.000000e+00,0.000000e+00
5,1.703101e+10,-3.813938e+00,4.574656e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-3.919983e+00,2.010000e+03,3.423233e+13,0.000000e+00,0.000000e+00
6,1.703101e+10,-3.675388e+00,4.489579e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.956076e+00,2.010000e+03,3.423233e+13,0.000000e+00,0.000000e+00


In [11]:
reg1<- lm(FA~  LINC + BlkD + WhtD + HispD, data = test)
summary(reg1)


Call:
lm(formula = FA ~ LINC + BlkD + WhtD + HispD, data = test)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.98402 -0.22522  0.01009  0.21823  1.31985 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -3.23368    0.15061 -21.471  < 2e-16 ***
LINC        -0.10337    0.03146  -3.286 0.001038 ** 
BlkD         0.28102    0.02919   9.626  < 2e-16 ***
WhtD         0.01245    0.02564   0.485 0.627414    
HispD       -0.08295    0.02377  -3.490 0.000496 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3165 on 1575 degrees of freedom
Multiple R-squared:  0.2067,	Adjusted R-squared:  0.2047 
F-statistic: 102.6 on 4 and 1575 DF,  p-value: < 2.2e-16


### First, a pooled regression of all of our observations

In [12]:
reg1<- lm(FA~  LINC + BlkD + WhtD + HispD + FRisk + TractYear, data = test)
summary(reg1)


Call:
lm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk + TractYear, 
    data = test)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.93336 -0.21383  0.01646  0.20752  1.04824 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.700e+01  8.566e+00   1.984  0.04741 *  
LINC        -9.315e-02  3.072e-02  -3.033  0.00246 ** 
BlkD         2.151e-01  2.867e-02   7.503 1.04e-13 ***
WhtD         1.213e-02  2.468e-02   0.492  0.62311    
HispD       -1.048e-01  2.297e-02  -4.563 5.43e-06 ***
FRisk        3.021e-01  2.644e-02  11.426  < 2e-16 ***
TractYear   -5.920e-13  2.508e-13  -2.361  0.01837 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3038 on 1573 degrees of freedom
Multiple R-squared:  0.2702,	Adjusted R-squared:  0.2674 
F-statistic: 97.06 on 6 and 1573 DF,  p-value: < 2.2e-16


In [13]:
reg1<- lm(FA~  LINC + BlkD + WhtD + HispD + FRisk + TractYear, data = test)
summary(reg1)


Call:
lm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk + TractYear, 
    data = test)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.93336 -0.21383  0.01646  0.20752  1.04824 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.700e+01  8.566e+00   1.984  0.04741 *  
LINC        -9.315e-02  3.072e-02  -3.033  0.00246 ** 
BlkD         2.151e-01  2.867e-02   7.503 1.04e-13 ***
WhtD         1.213e-02  2.468e-02   0.492  0.62311    
HispD       -1.048e-01  2.297e-02  -4.563 5.43e-06 ***
FRisk        3.021e-01  2.644e-02  11.426  < 2e-16 ***
TractYear   -5.920e-13  2.508e-13  -2.361  0.01837 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3038 on 1573 degrees of freedom
Multiple R-squared:  0.2702,	Adjusted R-squared:  0.2674 
F-statistic: 97.06 on 6 and 1573 DF,  p-value: < 2.2e-16


### If we add spatially lagged food access, the regression changes.

In [14]:
reg2<- lm(FA~  LINC + BlkD + WhtD + HispD + FRisk + FA_W, data = test)
summary(reg2)


Call:
lm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk + FA_W, 
    data = test)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.81700 -0.15175 -0.00063  0.14762  1.04145 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.003939   0.144032  -0.027   0.9782    
LINC        -0.046202   0.022927  -2.015   0.0441 *  
BlkD         0.001760   0.022542   0.078   0.9378    
WhtD         0.029269   0.018642   1.570   0.1166    
HispD       -0.044471   0.017451  -2.548   0.0109 *  
FRisk        0.175746   0.018520   9.489   <2e-16 ***
FA_W         0.944552   0.027489  34.361   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.23 on 1573 degrees of freedom
Multiple R-squared:  0.5816,	Adjusted R-squared:   0.58 
F-statistic: 364.5 on 6 and 1573 DF,  p-value: < 2.2e-16


### In a pooled OLS, with and without spatial lag, the results are sensitive and lead to differing conclusions.

Instead, let's set up a panel model to test fixed and random effects for each year and census tract. First, we set up the index as a data frame.

In [29]:
index1 <- pdata.frame(test, index=c("geoid10", "Year"))

### Fixed Effect model (without spatial effects)

In [30]:
fixed <-plm(FA~  LINC + BlkD + WhtD + HispD + FRisk, data=index1, model="within")
summary(fixed)

Oneway (individual) effect Within Model

Call:
plm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk, data = index1, 
    model = "within")

Balanced Panel: n=790, T=2, N=1580

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-0.4670 -0.0277  0.0000  0.0277  0.4670 

Coefficients :
        Estimate Std. Error t-value  Pr(>|t|)    
LINC  -0.0078086  0.0192142 -0.4064   0.68456    
BlkD   0.0644375  0.0351581  1.8328   0.06721 .  
WhtD   0.0146025  0.0169567  0.8612   0.38941    
HispD -0.0342703  0.0206100 -1.6628   0.09675 .  
FRisk  0.0690768  0.0124074  5.5674 3.551e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    12.379
Residual Sum of Squares: 11.774
R-Squared:      0.048824
Adj. R-Squared: 0.024257
F-statistic: 8.05878 on 5 and 785 DF, p-value: 2.0018e-07

In [17]:
fixedLag <-plm(FA~  LINC + BlkD + WhtD + HispD + FRisk + FA_W, data=index1, model="within")
summary(fixedLag)

Oneway (individual) effect Within Model

Call:
plm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk + FA_W, 
    data = index1, model = "within")

Balanced Panel: n=790, T=2, N=1580

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
 -0.429  -0.022   0.000   0.022   0.429 

Coefficients :
        Estimate Std. Error t-value Pr(>|t|)    
LINC   0.0113430  0.0149355  0.7595   0.4478    
BlkD   0.0306106  0.0273260  1.1202   0.2630    
WhtD  -0.0041661  0.0131856 -0.3160   0.7521    
HispD -0.0158964  0.0160154 -0.9926   0.3212    
FRisk  0.0129994  0.0099386  1.3080   0.1913    
FA_W   0.9760928  0.0428324 22.7886   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    12.379
Residual Sum of Squares: 7.0828
R-Squared:      0.42783
Adj. R-Squared: 0.21229
F-statistic: 97.7035 on 6 and 784 DF, p-value: < 2.22e-16

In [18]:
fixedC <-plm(FA~  LINC + BlkD + WhtD + HispD + FRisk + TractYear, data=index1, model="within")
summary(fixedC)


Oneway (individual) effect Within Model

Call:
plm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk + TractYear, 
    data = index1, model = "within")

Balanced Panel: n=790, T=2, N=1580

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-0.4690 -0.0265  0.0000  0.0265  0.4690 

Coefficients :
             Estimate  Std. Error t-value  Pr(>|t|)    
LINC      -1.4212e-02  1.9951e-02 -0.7123   0.47647    
BlkD       6.5860e-02  3.5169e-02  1.8727   0.06149 .  
WhtD       1.1474e-02  1.7156e-02  0.6688   0.50380    
HispD     -3.6480e-02  2.0688e-02 -1.7633   0.07823 .  
FRisk      6.0875e-02  1.4196e-02  4.2882 2.026e-05 ***
TractYear  1.3156e-13  1.1073e-13  1.1881   0.23515    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    12.379
Residual Sum of Squares: 11.753
R-Squared:      0.050533
Adj. R-Squared: 0.025075
F-statistic: 6.95444 on 6 and 784 DF, p-value: 3.2325e-07

In [19]:
fixedCLag <-plm(FA~  LINC + BlkD + WhtD + HispD + FRisk + TractYear + FA_W, data=index1, model="within")
summary(fixedCLag)


Oneway (individual) effect Within Model

Call:
plm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk + TractYear + 
    FA_W, data = index1, model = "within")

Balanced Panel: n=790, T=2, N=1580

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
 -0.429  -0.022   0.000   0.022   0.429 

Coefficients :
             Estimate  Std. Error t-value Pr(>|t|)    
LINC       1.1316e-02  1.5538e-02  0.7283   0.4667    
BlkD       3.0617e-02  2.7363e-02  1.1189   0.2635    
WhtD      -4.1787e-03  1.3344e-02 -0.3132   0.7542    
HispD     -1.5906e-02  1.6096e-02 -0.9882   0.3234    
FRisk      1.2966e-02  1.1227e-02  1.1549   0.2485    
TractYear  5.4537e-16  8.6204e-14  0.0063   0.9950    
FA_W       9.7607e-01  4.2956e-02 22.7227   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    12.379
Residual Sum of Squares: 7.0828
R-Squared:      0.42783
Adj. R-Squared: 0.21202
F-statistic: 83.6391 on 7 and 783 DF, p-value: < 2.22e-16

## Let's conduct some tests on the fixed effects results. 
As a DID setup, we know the model is subject to multiple challenges that can lead to erroneous conclusions.

### Breusch-Pagan LM Test
A Breusch-Pagan LM test suggests that there are signficant individual effects at the unit level. Because it fails, 
we must consider a Random Effects model. 

In [20]:
plmtest(fixed, type=c("bp")) 


	Lagrange Multiplier Test - (Breusch-Pagan)

data:  FA ~ LINC + BlkD + WhtD + HispD + FRisk
chisq = 469.12, df = 1, p-value < 2.2e-16
alternative hypothesis: significant effects


In [21]:
plmtest(fixed2, type=c("bp")) 

ERROR: Error in plmtest(fixed2, type = c("bp")): object 'fixed2' not found


###  Tests for Cross-Sectional Dependence

Next we test for cross-sectional dependence, an area of concern for DID analysis. The fixed effect model fails.


In [22]:
pcdtest(fixed, test = c("lm")) #Breusch-Pagan LM test of independence (FAIL)
pcdtest(fixed, test = c("cd")) # Pesaran CD test (PASS)


	Breusch-Pagan LM test for cross-sectional dependence in panels

data:  formula
chisq = 623310, df = 311660, p-value < 2.2e-16
alternative hypothesis: cross-sectional dependence



	Pesaran CD test for cross-sectional dependence in panels

data:  formula
z = 13.231, p-value < 2.2e-16
alternative hypothesis: cross-sectional dependence


### Test for serial correlation 
The Breusch-Godfrey/Wooldridge test for serial correlation also fails.


In [23]:
pbgtest(fixed)


	Breusch-Godfrey/Wooldridge test for serial correlation in panel models

data:  FA ~ LINC + BlkD + WhtD + HispD + FRisk
chisq = 707.12, df = 2, p-value < 2.2e-16
alternative hypothesis: serial correlation in idiosyncratic errors


In [24]:
pbgtest(fixedC)


	Breusch-Godfrey/Wooldridge test for serial correlation in panel models

data:  FA ~ LINC + BlkD + WhtD + HispD + FRisk + TractYear
chisq = 706.97, df = 2, p-value < 2.2e-16
alternative hypothesis: serial correlation in idiosyncratic errors


### Test for heteroskedasticity
Finally, we test for heteroskedasticity. It fails, but not as much as previous tests.

In [25]:
bptest(fixed, studentize=F)


	Breusch-Pagan test

data:  fixed
BP = 19.611, df = 5, p-value = 0.001478


## Random Effects Model
Next, we try a Random Effects model to account for individual heterogeneity. 

In [26]:
random<-plm(FA~  LINC + BlkD + WhtD + HispD + FRisk, data=index1, model="random")
summary(random)

Oneway (individual) effect Random Effect Model 
   (Swamy-Arora's transformation)

Call:
plm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk, data = index1, 
    model = "random")

Balanced Panel: n=790, T=2, N=1580

Effects:
                  var std.dev share
idiosyncratic 0.01500 0.12247 0.176
individual    0.07033 0.26519 0.824
theta:  0.6896  

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-0.4870 -0.0824  0.0011  0.0785  0.7410 

Coefficients :
             Estimate Std. Error  t-value  Pr(>|t|)    
(Intercept) -3.555116   0.090205 -39.4113 < 2.2e-16 ***
LINC        -0.030776   0.018671  -1.6483 0.0994837 .  
BlkD         0.215982   0.022606   9.5542 < 2.2e-16 ***
WhtD        -0.010666   0.015701  -0.6793 0.4970236    
HispD       -0.062607   0.017848  -3.5079 0.0004644 ***
FRisk        0.106227   0.012491   8.5040 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    30.356
Residual Sum of Squares: 25.75
R-Squared:   

In [27]:
randomC<-plm(FA~  LINC + BlkD + WhtD + HispD + FRisk + TractYear, data=index1, model="random")
summary(randomC)

Oneway (individual) effect Random Effect Model 
   (Swamy-Arora's transformation)

Call:
plm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk + TractYear, 
    data = index1, model = "random")

Balanced Panel: n=790, T=2, N=1580

Effects:
                  var std.dev share
idiosyncratic 0.01499 0.12244 0.181
individual    0.06780 0.26038 0.819
theta:  0.6845  

Residuals :
    Min.  1st Qu.   Median  3rd Qu.     Max. 
-0.48900 -0.08370  0.00061  0.07990  0.74400 

Coefficients :
               Estimate  Std. Error t-value  Pr(>|t|)    
(Intercept) -5.8391e+00  3.8979e+00 -1.4980 0.1343360    
LINC        -3.4298e-02  1.9429e-02 -1.7653 0.0777084 .  
BlkD         2.1632e-01  2.2663e-02  9.5450 < 2.2e-16 ***
WhtD        -1.1876e-02  1.5915e-02 -0.7462 0.4556439    
HispD       -6.3962e-02  1.7984e-02 -3.5566 0.0003868 ***
FRisk        1.0311e-01  1.4387e-02  7.1668 1.176e-12 ***
TractYear    6.7165e-14  1.1446e-13  0.5868 0.5574380    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.

In [28]:
randomCLag<-plm(FA~  LINC + BlkD + WhtD + HispD + FRisk + TractYear + FA_W, data=index1, model="random")
summary(randomCLag)

Oneway (individual) effect Random Effect Model 
   (Swamy-Arora's transformation)

Call:
plm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk + TractYear + 
    FA_W, data = index1, model = "random")

Balanced Panel: n=790, T=2, N=1580

Effects:
                   var  std.dev share
idiosyncratic 0.009046 0.095109 0.188
individual    0.039000 0.197483 0.812
theta:  0.6776  

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-0.5550 -0.0609 -0.0036  0.0580  0.5370 

Coefficients :
               Estimate  Std. Error t-value  Pr(>|t|)    
(Intercept)  3.7954e+00  2.9986e+00  1.2657    0.2058    
LINC         2.1100e-03  1.4885e-02  0.1418    0.8873    
BlkD         2.3902e-02  1.8114e-02  1.3196    0.1872    
WhtD         4.5018e-03  1.2163e-02  0.3701    0.7113    
HispD       -2.0248e-02  1.3755e-02 -1.4720    0.1412    
FRisk        4.5340e-02  1.1145e-02  4.0682 4.973e-05 ***
TractYear   -1.1463e-13  8.7827e-14 -1.3052    0.1920    
FA_W         9.6706e-01  2.8295e-02 34.1777 < 2

## Diagnostics

In [29]:
# Test for heteroskedasticity 
bptest(random, studentize=F)

#Test for serial correlation 
pbgtest(random)

#Test for x-sectional dependence
pcdtest(random, test = c("lm")) #Breusch-Pagan LM test of independence 




	Breusch-Pagan test

data:  random
BP = 19.611, df = 5, p-value = 0.001478



	Breusch-Godfrey/Wooldridge test for serial correlation in panel models

data:  FA ~ LINC + BlkD + WhtD + HispD + FRisk
chisq = 198.8, df = 2, p-value < 2.2e-16
alternative hypothesis: serial correlation in idiosyncratic errors



	Breusch-Pagan LM test for cross-sectional dependence in panels

data:  formula
chisq = 623310, df = 311660, p-value < 2.2e-16
alternative hypothesis: cross-sectional dependence


In [30]:
# Test for heteroskedasticity 
bptest(randomC, studentize=F)

#Test for serial correlation 
pbgtest(randomC)

#Test for x-sectional dependence
pcdtest(randomC, test = c("lm")) #Breusch-Pagan LM test of independence




	Breusch-Pagan test

data:  randomC
BP = 16.329, df = 6, p-value = 0.01209



	Breusch-Godfrey/Wooldridge test for serial correlation in panel models

data:  FA ~ LINC + BlkD + WhtD + HispD + FRisk + TractYear
chisq = 200.41, df = 2, p-value < 2.2e-16
alternative hypothesis: serial correlation in idiosyncratic errors



	Breusch-Pagan LM test for cross-sectional dependence in panels

data:  formula
chisq = 623310, df = 311660, p-value < 2.2e-16
alternative hypothesis: cross-sectional dependence


## Refine Spatial Effects

In [31]:
##################################
## Create big W matrix for panel analysis
##################################

W_FA_q <- poly2nb(chiFR,queen=TRUE)


In [32]:
library(splm)
library(Matrix)

In [33]:
length(W_FA_q)

#convert weight to weight-list object
wr<- nb2listw(W_FA_q)

#convert to sparse matrix
w1<- listw2dgCMatrix(wr)

[1] 790

In [34]:
#create time periods, validate
time_id<- c("2010","2014")
tperiods <- length(time_id)
tperiods

[1] 2

In [35]:
#calc Kronecker product using sparse matrix with Diagonal(tperiods)
I.T <- Diagonal(tperiods)

bigWmat <- kronecker(I.T,w1)
str(bigWmat)

#convert back to list weights object
bigW <- mat2listw(bigWmat,style="W")
summary(bigW)


Formal class 'dgTMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:10368] 2 3 2 9 10 11 0 1 3 8 ...
  ..@ j       : int [1:10368] 0 0 1 1 1 1 2 2 2 2 ...
  ..@ Dim     : int [1:2] 1580 1580
  ..@ Dimnames:List of 2
  .. ..$ : NULL
  .. ..$ : NULL
  ..@ x       : num [1:10368] 0.167 0.167 0.167 0.143 0.167 ...
  ..@ factors : list()
Characteristics of weights list object:
Neighbour list object:
Number of regions: 1580 
Number of nonzero links: 10368 
Percentage nonzero weights: 0.4153181 
Average number of links: 6.562025 
Link number distribution:

  1   2   3   4   5   6   7   8   9  10  11  12  13  14  18 
  2  10  60 114 238 314 420 268  70  44  16  18   2   2   2 
2 least connected regions:
143 933 with 1 link
2 most connected regions:
783 1573 with 18 links

Weights style: W 
Weights constants summary:
     n      nn   S0       S1       S2
W 1580 2496400 1580 509.2441 6460.457


In [36]:
test$LINC_W <- lag.listw(bigW, test$LINC)
test$WhtD_W <- lag.listw(bigW, test$WhtD)
test$BlkD_W <- lag.listw(bigW, test$BlkD)
test$HispD_W <- lag.listw(bigW, test$HispD)
head(test)

,geoid10,FA,LINC,FRisk,BlkD,WhtD,HispD,FA_W,Year,TractYear,YearD,FD,LINC_W,WhtD_W,BlkD_W,HispD_W
1,1.703101e+10,-4.086695e+00,4.476614e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-4.170998e+00,2.010000e+03,3.423233e+13,0.000000e+00,0.000000e+00,4.578690e+00,5.000000e-01,0.000000e+00,0.000000e+00
2,1.703101e+10,-4.225976e+00,4.534597e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.994378e+00,2.010000e+03,3.423233e+13,0.000000e+00,0.000000e+00,4.584895e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,1.703101e+10,-4.231131e+00,4.534597e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-4.103940e+00,2.010000e+03,3.423233e+13,0.000000e+00,0.000000e+00,4.562828e+00,1.666667e-01,1.666667e-01,0.000000e+00
4,1.703101e+10,-4.110865e+00,4.622784e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-3.993752e+00,2.010000e+03,3.423233e+13,0.000000e+00,0.000000e+00,4.540560e+00,1.666667e-01,1.666667e-01,0.000000e+00
5,1.703101e+10,-3.813938e+00,4.574656e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-3.919983e+00,2.010000e+03,3.423233e+13,0.000000e+00,0.000000e+00,4.564508e+00,8.333333e-01,0.000000e+00,0.000000e+00
6,1.703101e+10,-3.675388e+00,4.489579e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.956076e+00,2.010000e+03,3.423233e+13,0.000000e+00,0.000000e+00,4.569968e+00,7.500000e-01,0.000000e+00,0.000000e+00


In [37]:
NROW(test)

[1] 1580

In [38]:
index2 <- pdata.frame(test, index=c("geoid10", "Year"))

In [39]:
ctest1<- lm.LMtests(lm(FA~  LINC + BlkD + WhtD + HispD + FRisk, data = index1),bigW,test="all")
summary(ctest1)

	Lagrange multiplier diagnostics for spatial dependence
data:  
model: lm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk, data =
index1)
weights: bigW
 
       statistic parameter   p.value    
LMerr   286.8980         1 < 2.2e-16 ***
LMlag   268.9043         1 < 2.2e-16 ***
RLMerr   26.7203         1 2.351e-07 ***
RLMlag    8.7267         1  0.003136 ** 
SARMA   295.6247         2 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [40]:
ctest1<- lm.LMtests(lm(FA~  LINC + BlkD + WhtD + HispD + FRisk + TractYear, data = index1),bigW,test="all")
summary(ctest1)

	Lagrange multiplier diagnostics for spatial dependence
data:  
model: lm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk +
TractYear, data = index1)
weights: bigW
 
       statistic parameter   p.value    
LMerr   285.1204         1 < 2.2e-16 ***
LMlag   266.5828         1 < 2.2e-16 ***
RLMerr   27.1287         1 1.904e-07 ***
RLMlag    8.5911         1  0.003378 ** 
SARMA   293.7115         2 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

## Consistent estimates with SPML code

In [41]:
pooledSE<- spml(FA~  LINC + BlkD + WhtD + HispD + FRisk,data=index1,listw=wr,model="pooling", lag=TRUE,spatial.error="none")
summary(pooledSE)

ML panel with spatial lag and iid errors 

Call:
spreml(formula = formula, data = data, index = index, w = listw2mat(listw), 
    w2 = listw2mat(listw2), lag = lag, errors = errors, cl = cl)

Residuals:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  -3.60   -2.80   -2.55   -2.56   -2.33   -1.42 

Spatial autoregressive coefficient:
       Estimate Std. Error t-value  Pr(>|t|)    
lambda 0.705347   0.021827  32.316 < 2.2e-16 ***

Coefficients:
             Estimate Std. Error t-value  Pr(>|t|)    
(Intercept) -0.814972   0.111762 -7.2920 3.054e-13 ***
LINC        -0.062081   0.023344 -2.6593 0.0078294 ** 
BlkD         0.054858   0.022151  2.4766 0.0132643 *  
WhtD         0.023283   0.019026  1.2237 0.2210601    
HispD       -0.061198   0.017719 -3.4538 0.0005528 ***
FRisk        0.200747   0.018673 10.7506 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [158]:
pooledNSE<- spml(FA~  LINC + BlkD + WhtD + HispD + FRisk,data=index1,listw=wr,model="pooling", lag=FALSE,spatial.error="none")
summary(pooledNSE)

ML panel with iid errors 

Call:
spreml(formula = formula, data = data, index = index, w = listw2mat(listw), 
    w2 = listw2mat(listw2), lag = lag, errors = errors, cl = cl)

Residuals:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-0.9490 -0.2120  0.0133  0.0000  0.2100  1.0400 

Coefficients:
              Estimate Std. Error  t-value  Pr(>|t|)    
(Intercept) -3.2212841  0.1444849 -22.2949 < 2.2e-16 ***
LINC        -0.1064681  0.0301794  -3.5278  0.000419 ***
BlkD         0.2125689  0.0286360   7.4231 1.144e-13 ***
WhtD         0.0077438  0.0245971   0.3148  0.752894    
HispD       -0.1083539  0.0229072  -4.7301 2.244e-06 ***
FRisk        0.2767047  0.0241404  11.4623 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Spatial Lag Fixed Effects

In [58]:
fm=FA~  LINC + BlkD + WhtD + HispD + FRisk

In [45]:
fixedSE<- spml(fm,data=test,listw=wr,model="within", lag=TRUE,spatial.error="none")
summary(fixedSE)

Spatial panel fixed effects lag model
 

Call:
spml(formula = fm, data = test, listw = wr, model = "within", 
    lag = TRUE, spatial.error = "none")

Residuals:
     Min.   1st Qu.    Median   3rd Qu.      Max. 
-3.89e-01 -1.23e-02  2.22e-16  1.23e-02  3.89e-01 

Spatial autoregressive coefficient:
        Estimate Std. Error t-value  Pr(>|t|)    
lambda 0.9568135  0.0054646  175.09 < 2.2e-16 ***

Coefficients:
        Estimate Std. Error t-value  Pr(>|t|)    
LINC  -0.0164082  0.0052828 -3.1060  0.001896 ** 
BlkD   0.0052332  0.0048913  1.0699  0.284663    
WhtD   0.0022016  0.0043368  0.5077  0.611695    
HispD -0.0021234  0.0040306 -0.5268  0.598315    
FRisk  0.0363036  0.0042557  8.5306 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [161]:
coeftest(fixedNSE, vcovHC(fixedNSE, type = "HC1"))


t test of coefficients:

        Estimate Std. Error t value  Pr(>|t|)    
LINC  -0.0078086  0.0167314 -0.4667   0.64084    
BlkD   0.0644375  0.0601445  1.0714   0.28433    
WhtD   0.0146025  0.0138898  1.0513   0.29344    
HispD -0.0342703  0.0129782 -2.6406   0.00844 ** 
FRisk  0.0690768  0.0132666  5.2068 2.456e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [144]:
fixedNSE<- plm(fm,data=index1,model="within")
summary(fixedNSE)

Oneway (individual) effect Within Model

Call:
plm(formula = fm, data = index1, model = "within")

Balanced Panel: n=790, T=2, N=1580

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-0.4670 -0.0277  0.0000  0.0277  0.4670 

Coefficients :
        Estimate Std. Error t-value  Pr(>|t|)    
LINC  -0.0078086  0.0192142 -0.4064   0.68456    
BlkD   0.0644375  0.0351581  1.8328   0.06721 .  
WhtD   0.0146025  0.0169567  0.8612   0.38941    
HispD -0.0342703  0.0206100 -1.6628   0.09675 .  
FRisk  0.0690768  0.0124074  5.5674 3.551e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    12.379
Residual Sum of Squares: 11.774
R-Squared:      0.048824
Adj. R-Squared: 0.024257
F-statistic: 8.05878 on 5 and 785 DF, p-value: 2.0018e-07

In [183]:
fixedNSE2<- plm(FA~  LINC + BlkD + WhtD + HispD + FRisk, data=index1,model="within")
summary(fixedNSE2)


Oneway (individual) effect Within Model

Call:
plm(formula = FA ~ LINC + BlkD + WhtD + HispD + FRisk, data = index1, 
    model = "within")

Balanced Panel: n=790, T=2, N=1580

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-0.4670 -0.0277  0.0000  0.0277  0.4670 

Coefficients :
        Estimate Std. Error t-value  Pr(>|t|)    
LINC  -0.0078086  0.0192142 -0.4064   0.68456    
BlkD   0.0644375  0.0351581  1.8328   0.06721 .  
WhtD   0.0146025  0.0169567  0.8612   0.38941    
HispD -0.0342703  0.0206100 -1.6628   0.09675 .  
FRisk  0.0690768  0.0124074  5.5674 3.551e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    12.379
Residual Sum of Squares: 11.774
R-Squared:      0.048824
Adj. R-Squared: 0.024257
F-statistic: 8.05878 on 5 and 785 DF, p-value: 2.0018e-07

In [167]:
fixedSEtime<- spml(fm,data=test,listw=wr,model="within",effect="time",
                lag=TRUE,spatial.error="none")
summary(fixedSEtime)

Spatial panel fixed effects lag model
 

Call:
spml(formula = fm, data = test, listw = wr, model = "within", 
    effect = "time", lag = TRUE, spatial.error = "none")

Residuals:
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-0.3700 -0.0723 -0.0160  0.0439  0.9490 

Spatial autoregressive coefficient:
       Estimate Std. Error t-value  Pr(>|t|)    
lambda 0.740504   0.020746  35.694 < 2.2e-16 ***

Coefficients:
        Estimate Std. Error t-value  Pr(>|t|)    
LINC  -0.0414729  0.0129513 -3.2022 0.0013637 ** 
BlkD   0.0226755  0.0124638  1.8193 0.0688635 .  
WhtD   0.0021237  0.0105432  0.2014 0.8403663    
HispD -0.0336087  0.0098468 -3.4131 0.0006422 ***
FRisk  0.0645053  0.0106053  6.0824 1.184e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [166]:
fixedNSEtime<-plm(fm,data=index1,model="within",effect="time")
summary(fixedNSEtime)

Oneway (time) effect Within Model

Call:
plm(formula = fm, data = index1, effect = "time", model = "within")

Balanced Panel: n=790, T=2, N=1580

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-0.9330 -0.2130  0.0161  0.2070  1.0500 

Coefficients :
       Estimate Std. Error t-value  Pr(>|t|)    
LINC  -0.092700   0.030711 -3.0184  0.002582 ** 
BlkD   0.214833   0.028660  7.4958 1.095e-13 ***
WhtD   0.012294   0.024676  0.4982  0.618385    
HispD -0.104852   0.022960 -4.5668 5.338e-06 ***
FRisk  0.303063   0.026451 11.4574 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    198.73
Residual Sum of Squares: 145.15
R-Squared:      0.26965
Adj. R-Squared: 0.26845
F-statistic: 116.15 on 5 and 1573 DF, p-value: < 2.22e-16

In [140]:
fixedSEtime<- spml(fm,data=test,listw=wr,model="within",effect="time",
                lag=TRUE,spatial.error="none", method="w2sls",HESS=FALSE)
summary(fixedSEtime)

ERROR: Error in jacobianSetup(method, env, con, pre_eig = con$pre_eig, trs = trs1, : ...

Unknown method



In [63]:
fixedSEind<- spml(fm,data=test,listw=wr,model="within",effect="individual",
                lag=TRUE,spatial.error="none", method="LU",HESS=FALSE)
summary(fixedSEind)

Spatial panel fixed effects lag model
 

Call:
spml(formula = fm, data = test, listw = wr, model = "within", 
    effect = "individual", lag = TRUE, spatial.error = "none", 
    method = "LU", HESS = FALSE)

Residuals:
     Min.   1st Qu.    Median   3rd Qu.      Max. 
-3.89e-01 -1.23e-02  2.22e-16  1.23e-02  3.89e-01 

Spatial autoregressive coefficient:
        Estimate Std. Error t-value  Pr(>|t|)    
lambda 0.9568135  0.0054646  175.09 < 2.2e-16 ***

Coefficients:
        Estimate Std. Error t-value  Pr(>|t|)    
LINC  -0.0164082  0.0052828 -3.1060  0.001896 ** 
BlkD   0.0052332  0.0048913  1.0699  0.284663    
WhtD   0.0022016  0.0043368  0.5077  0.611695    
HispD -0.0021234  0.0040306 -0.5268  0.598315    
FRisk  0.0363036  0.0042557  8.5306 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [138]:
fixedGM2s<- spgm(fm, data=test, listw=wr, model=("within"), lag = TRUE, spatial.error=FALSE, method = "w2sls")
summary(fixedGM2s)


Call:spgm(formula = fm, data = test, listw = wr, model = ("within"), 
    lag = TRUE, spatial.error = FALSE, method = "w2sls")

Residuals:
        Min          1Q      Median          3Q         Max 
-3.8961e-01 -1.0379e-02  5.5511e-17  1.0379e-02  3.8961e-01 

Coefficients: 
          Estimate  Std. Error  t value  Pr(>|t|)
lambda  0.97057593  0.00618288 156.9781 < 2.2e-16
LINC   -0.01497644  0.00747706  -2.0030   0.04518
BlkD    0.00107322  0.00708804   0.1514   0.87965
WhtD    0.00180806  0.00612964   0.2950   0.76802
HispD  -0.00043411  0.00573628  -0.0757   0.93968
FRisk   0.03202535  0.00616681   5.1932 2.067e-07

Residual variance (sigma squared): 0.0013991, (sigma: 0.037405)


### Spatial dependence tests on panel data
This tests are panel versions of the locally robust LM tests of Anselin et al. (1996), based on
a pooling assumption: i.e., they do not allow for any kind of individual effect. Therefore it is
advisable to employ a within transformation whenever individual effects cannot be ruled out. (Gianfranco 2016)

In [67]:
slmtest(fm, data=index1, listw =wr, test="lml")



	LM test for spatial lag dependence

data:  formula 
LM = 847.07, df = 1, p-value < 2.2e-16
alternative hypothesis: spatial lag dependence


In [68]:
## model on within-transformed (time-demeaned) data,
## eliminates individual effects
slmtest(fm, data=index1, listw =wr,  test="lml",model="within")


	LM test for spatial lag dependence

data:  formula (within transformation)
LM = 797.14, df = 1, p-value < 2.2e-16
alternative hypothesis: spatial lag dependence


In [69]:
slmtest(fm, data=index1, listw =wr, test="lme")



	LM test for spatial error dependence

data:  formula 
LM = 912.3, df = 1, p-value < 2.2e-16
alternative hypothesis: spatial error dependence


In [70]:
## model on within-transformed (time-demeaned) data,
## eliminates individual effects
slmtest(fm, data=index1, listw =wr,  test="lme",model="within")


	LM test for spatial error dependence

data:  formula (within transformation)
LM = 754.92, df = 1, p-value < 2.2e-16
alternative hypothesis: spatial error dependence


### Hausmann Test

In [43]:
fm=FA~  LINC + BlkD + WhtD + HispD + FRisk
test1<-sphtest(fm,data=test, listw = wr, spatial.model = "error", method="GM")
test1

Note: method with signature ‘sparseMatrix#matrix’ chosen for function ‘%*%’,
 target signature ‘dgTMatrix#matrix’.
 "TsparseMatrix#ANY" would also be valid
Note: method with signature ‘Matrix#numLike’ chosen for function ‘%*%’,
 target signature ‘dgTMatrix#numeric’.
 "TsparseMatrix#ANY" would also be valid



	Hausman test for spatial models

data:  x
chisq = 3.6215, df = 5, p-value = 0.6051
alternative hypothesis: one model is inconsistent


In [179]:
fm=FA~  LINC + BlkD + WhtD + HispD + FRisk
test1<-sphtest(fm,data=test, listw = wr, spatial.model = "sarar", method="GM")
test1


	Hausman test for spatial models

data:  x
chisq = 19.098, df = 6, p-value = 0.004001
alternative hypothesis: one model is inconsistent


In [84]:
fm=FA~  LINC + BlkD + WhtD + HispD + FRisk
test1<-sphtest(fm,data=index1, listw = wr, spatial.model = "lag")
test1


	Hausman test for spatial models

data:  x
chisq = 41.711, df = 5, p-value = 6.74e-08
alternative hypothesis: one model is inconsistent


In [170]:
mod1<- spgm(fm, data=test, listw = wr, model = "random", moments="initial")

mod2<- spgm(fm, data=test, listw = wr, model = "within")

test2<-sphtest(mod1, mod2)
test2


	Hausman test for spatial models

data:  fm
chisq = 3.6034, df = 5, p-value = 0.6078
alternative hypothesis: one model is inconsistent


In [99]:
test3<-sphtest(fm,data=index1, listw = wr, spatial.model = "lag",model="within")
test3


	Hausman test for spatial models

data:  x
chisq = 41.711, df = 5, p-value = 6.74e-08
alternative hypothesis: one model is inconsistent


In [100]:
test4<-sphtest(fm,data=index1, listw = wr, model="random")
test4


	Hausman test for spatial models

data:  x
chisq = 41.711, df = 5, p-value = 6.74e-08
alternative hypothesis: one model is inconsistent


## ML Spatial Lag Random Effects

In [48]:
fm=FA~  LINC + BlkD + WhtD + HispD + FRisk

In [89]:
randomSML<- spml(fm,data=index1,listw=wr,model="random",effect="individual",
                lag=TRUE,spatial.error="none", method="LU")
summary(randomSML)

ML panel with spatial lag, random effects 

Call:
spreml(formula = formula, data = data, index = index, w = listw2mat(listw), 
    w2 = listw2mat(listw2), lag = lag, errors = errors, cl = cl, 
    method = "LU")

Residuals:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  -3.63   -2.83   -2.58   -2.58   -2.34   -1.32 

Error variance parameters:
    Estimate Std. Error t-value  Pr(>|t|)    
phi  5.06629    0.40316  12.566 < 2.2e-16 ***

Spatial autoregressive coefficient:
       Estimate Std. Error t-value  Pr(>|t|)    
lambda 0.708809   0.022136  32.021 < 2.2e-16 ***

Coefficients:
              Estimate Std. Error  t-value  Pr(>|t|)    
(Intercept) -1.0334549  0.0690211 -14.9730 < 2.2e-16 ***
LINC        -0.0089207  0.0142797  -0.6247   0.53216    
BlkD         0.0746132  0.0174819   4.2680 1.972e-05 ***
WhtD        -0.0015918  0.0120211  -0.1324   0.89466    
HispD       -0.0320473  0.0137269  -2.3346   0.01956 *  
FRisk        0.0534015  0.0095307   5.6031 2.106e-08 ***
---
Signi

In [135]:
randomSE1<- spml(fm,data=index1,listw=wr,model="random",effect="individual",
                lag=TRUE,spatial.error="none", method="ec2sls")
summary(randomSE1)

ML panel with spatial lag, random effects 

Call:
spreml(formula = formula, data = data, index = index, w = listw2mat(listw), 
    w2 = listw2mat(listw2), lag = lag, errors = errors, cl = cl, 
    method = "ec2sls")

Residuals:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  -3.63   -2.83   -2.58   -2.58   -2.34   -1.32 

Error variance parameters:
    Estimate Std. Error t-value  Pr(>|t|)    
phi  5.06629    0.40316  12.566 < 2.2e-16 ***

Spatial autoregressive coefficient:
       Estimate Std. Error t-value  Pr(>|t|)    
lambda 0.708809   0.022136  32.021 < 2.2e-16 ***

Coefficients:
              Estimate Std. Error  t-value  Pr(>|t|)    
(Intercept) -1.0334549  0.0690211 -14.9730 < 2.2e-16 ***
LINC        -0.0089207  0.0142797  -0.6247   0.53216    
BlkD         0.0746132  0.0174819   4.2680 1.972e-05 ***
WhtD        -0.0015918  0.0120211  -0.1324   0.89466    
HispD       -0.0320473  0.0137269  -2.3346   0.01956 *  
FRisk        0.0534015  0.0095307   5.6031 2.106e-08 ***
---
S

In [44]:
randomNSE<- spml(fm,data=index1,listw=wr,model="random",effect="individual",
                lag=FALSE,spatial.error="none", method="ec2sls")
summary(randomNSE)

ML panel with , random effects 

Call:
spreml(formula = formula, data = data, index = index, w = listw2mat(listw), 
    w2 = listw2mat(listw2), lag = lag, errors = errors, cl = cl, 
    method = "ec2sls")

Residuals:
    Min.  1st Qu.   Median  3rd Qu.     Max. 
-0.98900 -0.22300  0.00891  0.21200  1.21000 

Error variance parameters:
    Estimate Std. Error t-value  Pr(>|t|)    
phi  5.21234    0.41937  12.429 < 2.2e-16 ***

Coefficients:
             Estimate Std. Error  t-value  Pr(>|t|)    
(Intercept) -3.562451   0.088434 -40.2839 < 2.2e-16 ***
LINC        -0.028988   0.018293  -1.5847 0.1130330    
BlkD         0.213073   0.022487   9.4753 < 2.2e-16 ***
WhtD        -0.010386   0.015406  -0.6741 0.5002172    
HispD       -0.061295   0.017620  -3.4787 0.0005039 ***
FRisk        0.103519   0.012199   8.4860 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [173]:
testFR<-sphtest(fixedSE, randomSE)
testFR


	Hausman test for spatial models

data:  fm
chisq = 41.711, df = 5, p-value = 6.74e-08
alternative hypothesis: one model is inconsistent


In [130]:
randomLagML<- spml(fm,data=index1,listw=wr,model="random",
                lag=TRUE,spatial.error="none", method="ec2sls")
summary(randomLagML)

ML panel with spatial lag, random effects 

Call:
spreml(formula = formula, data = data, index = index, w = listw2mat(listw), 
    w2 = listw2mat(listw2), lag = lag, errors = errors, cl = cl, 
    method = "ec2sls")

Residuals:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  -3.63   -2.83   -2.58   -2.58   -2.34   -1.32 

Error variance parameters:
    Estimate Std. Error t-value  Pr(>|t|)    
phi  5.06629    0.40316  12.566 < 2.2e-16 ***

Spatial autoregressive coefficient:
       Estimate Std. Error t-value  Pr(>|t|)    
lambda 0.708809   0.022136  32.021 < 2.2e-16 ***

Coefficients:
              Estimate Std. Error  t-value  Pr(>|t|)    
(Intercept) -1.0334549  0.0690211 -14.9730 < 2.2e-16 ***
LINC        -0.0089207  0.0142797  -0.6247   0.53216    
BlkD         0.0746132  0.0174819   4.2680 1.972e-05 ***
WhtD        -0.0015918  0.0120211  -0.1324   0.89466    
HispD       -0.0320473  0.0137269  -2.3346   0.01956 *  
FRisk        0.0534015  0.0095307   5.6031 2.106e-08 ***
---
S

In [128]:
randomErrorGM2<- spgm(fm,data=index1,listw=wr,model="random",moments="fullweights",
                lag=FALSE,spatial.error=TRUE, method="ec2sls")
summary(randomErrorGM2)

 

Call:
spgm(formula = fm, data = index1, listw = wr, model = "random", 
    lag = FALSE, spatial.error = TRUE, moments = "fullweights", 
    method = "ec2sls")

Residuals:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.0100 -0.1930  0.0534  0.0574  0.3070  1.3600 

Coefficients:
              Estimate Std. Error  t-value  Pr(>|t|)    
(Intercept) -3.5234383  0.0926659 -38.0231 < 2.2e-16 ***
LINC        -0.0358240  0.0184144  -1.9454 0.0517232 .  
BlkD         0.0185698  0.0174059   1.0669 0.2860309    
WhtD         0.0028555  0.0147946   0.1930 0.8469531    
HispD       -0.0247072  0.0139819  -1.7671 0.0772140 .  
FRisk        0.0518566  0.0144389   3.5915 0.0003288 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [129]:
randomLagGM2<- spgm(fm,data=index1,listw=wr,model="random", moments="fullweights", lag = TRUE,method="ec2sls")
summary(randomLagGM2)

 

Call:
spgm(formula = fm, data = index1, listw = wr, model = "random", 
    lag = TRUE, moments = "fullweights", method = "ec2sls")

Residuals:
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-0.51200 -0.06260 -0.00651 -0.00739  0.01270  0.97900 

Coefficients:
              Estimate Std. Error  t-value  Pr(>|t|)    
lambda       0.9764569  0.0038048 256.6364 < 2.2e-16 ***
(Intercept) -0.0398315  0.0290852  -1.3695   0.17085    
LINC        -0.0118181  0.0055943  -2.1126   0.03464 *  
BlkD         0.0045988  0.0054751   0.8400   0.40093    
WhtD         0.0005851  0.0046603   0.1255   0.90009    
HispD       -0.0066648  0.0043414  -1.5352   0.12475    
FRisk        0.0260663  0.0048885   5.3322 9.704e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [131]:
randomLagGM1<- spgm(fm,data=index1,listw=wr,model="random", moments="initial", lag = TRUE,method="ec2sls")
summary(randomLagGM1)

 

Call:
spgm(formula = fm, data = index1, listw = wr, model = "random", 
    lag = TRUE, moments = "initial", method = "ec2sls")

Residuals:
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-0.51200 -0.06230 -0.00646 -0.00719  0.01290  0.97900 

Coefficients:
               Estimate  Std. Error  t-value  Pr(>|t|)    
lambda       0.97610385  0.00387464 251.9211 < 2.2e-16 ***
(Intercept) -0.04013729  0.02954864  -1.3583   0.17435    
LINC        -0.01208652  0.00567785  -2.1287   0.03328 *  
BlkD         0.00456636  0.00554915   0.8229   0.41057    
WhtD         0.00065847  0.00472626   0.1393   0.88920    
HispD       -0.00662803  0.00440314  -1.5053   0.13225    
FRisk        0.02646252  0.00495124   5.3446  9.06e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [132]:
randomLagGM1<- spgm(fm,data=index1,listw=wr,model="random", moments="initial", lag = FALSE,spatial.error=TRUE,method="ec2sls")
summary(randomLagGM1)

 

Call:
spgm(formula = fm, data = index1, listw = wr, model = "random", 
    lag = FALSE, spatial.error = TRUE, moments = "initial", method = "ec2sls")

Residuals:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.0100 -0.2000  0.0466  0.0506  0.3000  1.3500 

Coefficients:
              Estimate Std. Error  t-value  Pr(>|t|)    
(Intercept) -3.5134451  0.0927185 -37.8937 < 2.2e-16 ***
LINC        -0.0366962  0.0186085  -1.9720 0.0486082 *  
BlkD         0.0207296  0.0175967   1.1780 0.2387806    
WhtD         0.0028186  0.0149543   0.1885 0.8504989    
HispD       -0.0257518  0.0141314  -1.8223 0.0684075 .  
FRisk        0.0540500  0.0146074   3.7002 0.0002154 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
